### Washington State Crash Event Analysis
#### --- by 

In [155]:
import pandas as pd
import numpy as np
import regex as re

import requests
import asyncio
import json as js

import time

import os

pd.set_option('display.max_rows', 6)

#### Load Datasets

In [156]:
dir = os.path.abspath(os.path.dirname(os.getcwd())) + '/data/'

df_data = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0') 
df_data.event_zipcode = df_data.event_zipcode.astype(str)   # convert the default float type values into str
df_crash_type = pd.read_csv(dir + '/output/crash_type.csv')

df_data.shape

C:\Users\Chaconne\AppData\Local\Temp\ipykernel_30464\3630574484.py:3: DtypeWarning: Columns (22,23,24,302,304) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0')


(4132, 306)

In [157]:
df_data.shape

(4132, 306)

In [158]:
# drop rows which do not have a zipcode

has_no_zipcode = df_data.event_zipcode.map(lambda v : v == 'nan')
df_data = df_data[df_data.event_zipcode != 'nan']
df_data.shape

(4132, 306)

In [159]:
df_data.dzip = df_data.dzip.map(
    lambda n: 0 if n ==0 else 0 if pd.isna(n) else int(n)
)
df_data = df_data[df_data.dzip > 10000]
df_data.dzip = df_data.dzip.astype(str)
df_data.shape

(4100, 306)

##### Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

In [165]:
df_data['is_resident'] = df_data.index.map(
    lambda i: df_data.event_zipcode[i] == df_data.dzip[i]
)

df_data['is_driver'] = df_data.ptype.map(
    lambda t: t == 1
)

prop = len(df_data[(df_data.is_resident == True) & (
    df_data.is_driver == True)]) / float(len(df_data[df_data.is_driver == True]))

print('{prop:.4f}% of the drivers are from the same community where the accident occured'.format(prop = prop * 100))

23.7805% of the drivers are from the same community where the event occured


Based on our analysis, 23.7805% of the drivers are from the same community where the accident occured.

In [ ]:
df_crash_analysis = df_data[df_data.is_driver == True].crash_type

##### Are there differences in the types of crashes and behavior factors in those crashes among “residents” versus those deemed to be not “from” the area?

In [162]:
df_crash_type[4:10]

,type_index,info
4,4,Specifics Other
5,5,Specifics Unknown
6,6,Drive Off Road
7,7,Control/Traction Loss
8,8,"Avoid Collision With Vehicle, Pedestrian, Animal"
9,9,Specifics Other


In [163]:
crash_type_mapper = {
    # map a crash number to its actual meaning
    k: v for k,v in zip(
        df_crash_type['info'].map(
            lambda v: v.strip().split('=')[0].strip()
        ),
        df_crash_type['info'].map(
            lambda v: v.strip().split('=')[1].strip()
        ),
    )
}

IndexError: list index out of range

##### Analysis of Behavioral Factors

In [ ]:
df_data.loc[:, 'race1': 'race5'].apply(
    lambda col : sum(col.isnull())
)

race1    1556
race2    4051
race3    4100
race4    4107
race5    4107
dtype: int64

- The following columns are thought to indicate whether an involved person conducted risky behavior in the crash event.
- - restraintmisuse: valued 1 when there was a restraint misuse
- - helmetmisuse: valued 1 when there was a helmet misuse
- - 

##### Predictive Analysis of Risky Drivers